NOTE : Considering some of GEE's user limits and avoiding some system errors, such as "ERROR: user memory limit exceeded","TimeoutError", etc,we will process or download some data beforehand and upload it to GEE Assets, where it will be called up when needed.

Special reminder: the following variables or parameters need to be modified

Must be amended : "roi", "region", "elevation", "dataset_id", "assetID", "totalSamples"

May have to be modified (Depending on your actual situation) : 

In [ ]:
#### If you are trying to use geemap in coutries where Gooogle Services are blocked (e.g., China), 
#### you will need a VPN,then replace "10809" with your "proxy port number"to connect to Earth Engine servers.
#### Otherwise, you might encounter a connection timeout issue.

import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
#### Initializing GEE

import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
# 定义参数
year = '2020'
startDate = year + '-01-01'
endDate = year + '-12-31'

In [ ]:
#### Setting the boundaries of the study area.
#### Format: ee.Geometry.Rectangle(minLng, minLat, maxLng, maxLat)

roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
#### Defining variables
region = 'wuhan'

#### Calls for elevation data, which he has downloaded to GEE's Assets in "00--preparation.ipynb"
elevation = ee.Image("users/311605001111/hillshade_wuhan")

#### Visualisation parameters for Landsat8 images
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

# Define the relevant functions

In [ ]:
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('(B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7)/10000',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi)

def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

def maskSR_reverse(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    image_cloud = img.updateMask(mask.remap([0,1],[1,0]))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image_shadow = img.updateMask(ee.Terrain.hillshade(elevation,azimuth,zenith).remap([0,1],[1,0]))
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    image_outlier = img.updateMask(bands.gt(0).rename('outlier'))
    return ee.ImageCollection([image_cloud,image_shadow,image_outlier]).sum()

In [ ]:
## img refers to an image that has been indexed but not cloud-masked
def classified_image(img):
    image = maskSR(img).select(bands).classify(trainedClassifier).eq(1).remap([0,1],[1,2]).rename('waterclass').float()
    invalidPixel = maskSR_reverse(img).select('pixel_qa').gt(0).remap([0,1],[1,0]).rename('waterclass').float()
    class_image = ee.ImageCollection([invalidPixel,image]).sum()
    invalidPixels = class_image.eq(0).multiply(ee.Image.pixelArea()).divide(1e6)
    invalidarea = invalidPixels.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 200,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    region = class_image.gte(0).multiply(ee.Image.pixelArea()).divide(1e6)
    regionarea = region.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 200,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    rate = ee.Number(invalidarea).divide(ee.Number(regionarea)).multiply(100)
    return class_image.set({'system:id':img.get('system:id')}).set({'CLOUD_COVER':img.get('CLOUD_COVER')}).set({'invalid_percentage':rate})

def waterArea(image):
    classified_image = image.select('waterclass').eq(2).rename('waterclass')
    water_area = classified_image.multiply(ee.Image.pixelArea()).divide(1e6)
    waterarea = water_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi,
        'scale': 200,
        'maxPixels': 1e14,
        'tileScale': 2,
    })
    return image.set({'waterarea': waterarea.get('waterclass')}) 

def occurrence_Histogram(class_image):
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': roi,
        'scale': 30,
        'bestEffort': True,
        'tileScale': 2,
    })
    return class_image.set({'occurrence_HistogramCount': occurrence_HistogramCount.get('occurrence')})

def AutomaticCorrection_threshold(class_image):
    histogram = ee.List(ee.Dictionary(class_image.get('occurrence_HistogramCount')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(class_image.get('occurrence_HistogramCount')).get('bucketMeans'))
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    return class_image.set({'occurrence_threshold':occurrence_threshold})

def AutomaticCorrection_enhanced(class_image):
    basemap = ee.Image.constant(0).toFloat().updateMask(class_image.gte(0)).rename('waterclass')
    water = class_image.eq(2).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_no_data = occurrence.updateMask(class_image.eq(0).selfMask())
    occurrence_threshold = class_image.get('occurrence_threshold')
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water

# Generating RF classifiers

"totalSamples" denotes the training sample set downloaded to GEE's Assets in "01--Automatic training sample construction.ipynb",which needs to be adapted to your situation.

In [ ]:
totalSamples = ee.FeatureCollection('users/311605001111/' + region + '/' + region + '_9920')
label = 'waterclass'
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
trainedClassifier = ee.Classifier.smileRandomForest(150).train(totalSamples,label,bands)

# Water classification

Loading the image you want to correct

In [ ]:
ref_image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139036_20200718')
image = water_index(ref_image)
Map.addLayer(image.clip(roi),visParams,'raw image')
Map.addLayer(maskSR(image.clip(roi)),visParams,'mask image')
Map

Calculating relevant indicator data.

The existence of water in the image is a necessary condition for temporal correction.Therefore the "waterarea" must have more than 5km²

Only if the "invalid_percentage" is between 5% and 95% (5--95), and the "occurrence_threshold" is between 5% and 75% (5~75), the image needs to be corrected

In [ ]:
q = AutomaticCorrection_threshold(occurrence_Histogram(waterArea(classified_image(image))))
print('waterarea: ',q.get('waterarea').getInfo())
print('invalid_percentage: ',q.get('invalid_percentage').getInfo())
print('occurrence_threshold: ',q.get('occurrence_threshold').getInfo())

If the conditions are met, then implement the "temporal correction"

In [ ]:
enhance_image = AutomaticCorrection_enhanced(q)
Map.addLayer(enhance_image.selfMask(),{'palette':['blue']}, 'enhance water')

# publication quality maps

Introduction:

cartoee is a lightweight module to aid in creating publication quality maps from Earth Engine processing results without having to download data. The cartoee package does this by requesting png images from EE results (which are usually good enough for visualization) and cartopy is used to create the plots. Utility functions are available to create plot aesthetics such as gridlines or color bars. The notebook and the geemap cartoee module (cartoee.py) were contributed by Kel Markert. A huge thank you to him.

URL: https://geemap.org/cartoee/

In [ ]:
re = [91.3626, 34.6579, 89.8826, 35.8279]

ref_image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139036_20200718')
refer_image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139036_20200819')

In [ ]:
# 原始影像
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee

fig = plt.figure(figsize=(12, 8))
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}

# use cartoee to get a map
ax = cartoee.get_map(ref_image, region=re,vis_params=visParams)
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'raw image (20200718)', fontsize=24)

In [ ]:
# 分类影像（RF）
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee

fig = plt.figure(figsize=(12, 8))
vis = {'palette':['D4D4D4','blue'],'min':0,'max':1}
classify_image = maskSR(water_index(ref_image.clip(roi))).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')

# use cartoee to get a map
ax = cartoee.get_map(classify_image, region=re, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'classified image (w/o correction)', fontsize=24)

In [ ]:
# 分类影像（校正）
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee

fig = plt.figure(figsize=(12, 8))
vis = {'palette':['D4D4D4','blue'],'min':0,'max':1}

# use cartoee to get a map
ax = cartoee.get_map(enhance_image, region=re, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'classified image (with correction)', fontsize=24)